## Task 1. Artificial neural network implementation using a classification of choice of any image dataset 
In this project, we implement the AlexNet architecture for the classification of grapes.

In [74]:
import os
import cv2
import random
import time
import numpy as np 
import pandas as pd
import subprocess as sp
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import  layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPool2D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import gc

In [4]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

In [5]:
def get_model(numClasses, inputShape = (100,100,3)):

    model = models.Sequential()
    model.add(Conv2D(filters=64, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu', input_shape=(100,100,3)))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=192, kernel_size=(5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(layers.Dense(NumClasses, activation='softmax')) 
    return model


In [6]:
train_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Training"
test_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Test"

In [7]:
# Define image size and batch size
IMAGE_SIZE = (100, 100) 
BATCH_SIZE = 32

# Define paths to your dataset
TRAIN_DATA_PATH = train_path
VALIDATION_DATA_PATH = test_path

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_PATH,
    # shuffle=True,
    subset="validation",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_PATH,
    # shuffle=True,
    subset="training",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

NumClasses = train_generator.num_classes

Found 67692 images belonging to 131 classes.
Found 11335 images belonging to 131 classes.
Found 11353 images belonging to 131 classes.


In [8]:
print(f'Total Number of images in dataset: {train_generator.samples+ validation_generator.samples+ test_generator.samples} ')
print(f'Number of data samples for Training Dataset: {train_generator.samples} i.e. {train_generator.samples / (train_generator.samples+ validation_generator.samples+ test_generator.samples):.2f} of total dataset')
print(f'Number of data samples for Validation  Dataset: {validation_generator.samples} i.e. {validation_generator.samples / (train_generator.samples+ validation_generator.samples+ test_generator.samples):.2f} of total dataset')
print(f'Number of fata samples for Testing Dataset: {test_generator.samples} i.e. {test_generator.samples / (train_generator.samples+ validation_generator.samples+ test_generator.samples):.2f} of total dataset')

Total Number of images in dataset: 90380 
Number of data samples for Training Dataset: 67692 i.e. 0.75 of total dataset
Number of data samples for Validation  Dataset: 11335 i.e. 0.13 of total dataset
Number of fata samples for Testing Dataset: 11353 i.e. 0.13 of total dataset


In [9]:
EPOCHS = 10
model = get_model(NumClasses,(100,100,3))
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

#This is creating files with weights, so in case if something happens no need to wait again for model to train, can just use created weights
checkpoint_callback = ModelCheckpoint(filepath='model_weights_epoch_Alexnet{epoch:02d}.weights.h5', save_weights_only=True, save_freq='epoch')

C:\Users\Tomi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
history = model.fit(train_generator,validation_data = validation_generator,epochs=EPOCHS,callbacks=[checkpoint_callback,MyCustomCallback()])

Epoch 1/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 468s 221ms/step - accuracy: 0.9821 - loss: 0.0535 - val_accuracy: 0.9694 - val_loss: 0.0916
Epoch 2/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 446s 210ms/step - accuracy: 0.9862 - loss: 0.0412 - val_accuracy: 0.9809 - val_loss: 0.0655
Epoch 3/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 446s 210ms/step - accuracy: 0.9837 - loss: 0.0495 - val_accuracy: 0.9869 - val_loss: 0.0599
Epoch 4/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 449s 212ms/step - accuracy: 0.9848 - loss: 0.0458 - val_accuracy: 0.9871 - val_loss: 0.0408
Epoch 5/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 446s 210ms/step - accuracy: 0.9876 - loss: 0.0380 - val_accuracy: 0.9824 - val_loss: 0.0544
Epoch 6/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 454s 214ms/step - accuracy: 0.9884 - loss: 0.0345 - val_accuracy: 0.9837 - val_loss: 0.0504
Epoch 7/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 448s 212ms/step - accuracy: 0.9885 - loss: 0.0368 - val_accuracy: 0.9683 - val_loss: 0.0952
Epoch 8/10
2116/2116 ━━━━━━━━━━━━━━━━━━━━ 449s 212ms/step - ac

In [26]:
model.load_weights('model_weights_epoch_Alexnet10.weights.h5') 

In [27]:
# Evaluate AlexNet on Validation Data
scores = model.evaluate(validation_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

355/355 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9669 - loss: 0.1621
evaluate compile_metrics: 96.47%


In [64]:
#TESTING
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)

355/355 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.9709 - loss: 0.1480


Results as percentage: ~97%